# Assignment 1 - Product Output with Pydantic Structure Output

In [5]:
import os
from dotenv import load_dotenv
from pydantic import Field, BaseModel
from langchain_groq.chat_models import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [8]:
load_dotenv()

# print(os.getenv("OPENAI_API_KEY"))
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [9]:
class Product(BaseModel):
    product_name: str=Field(description="name of the product")
    product_details: str=Field(description="specifications and salient features of the product")
    tentative_price: float=Field(description="price of the product in indian ruppess i.e. INR")

In [10]:
output_parser = JsonOutputParser(pydantic_object=Product)

In [11]:
llm_2 = ChatGroq(model="gemma2-9b-it", max_tokens=1000, temperature=0.7)

In [12]:
prompt = ChatPromptTemplate(
    [
        ("system","You are an helpful assistant. Answer the user's query. {format_instructions}"),
        ("human", "{query}"),
    ],
    input_variables=["query"],
    partial_variables={"format_instructions":output_parser.get_format_instructions()}
)

prompt

ChatPromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"product_name": {"description": "name of the product", "title": "Product Name", "type": "string"}, "product_details": {"description": "specifications and salient features of the product", "title": "Product Details", "type": "string"}, "tentative_price": {"description": "price of the product in indian ruppess i.e. INR", "title": "Tentative Price", "type": "number"}}, "required": ["product_name", "product_details", "ten

In [13]:
chain = prompt|llm_2|output_parser
result=chain.invoke({"query":"how is samsung galaxy phones?"})

In [15]:
result

{'product_name': 'Samsung Galaxy Phones',
 'product_details': 'High-performance smartphones with advanced camera systems, sleek designs, and powerful processors.',
 'tentative_price': 30000}